In [1]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [2]:
import altair as alt

In [3]:
from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [4]:
pd.set_option("display.precision", 2)

In [5]:
df = pd.read_excel('OFFICIAGrantsOverviewandWalkthrough.xlsx')

In [6]:
df = to_snakecase(df)

In [7]:
df = df.rename(columns={'grant_type__formula,_competitive,_both,_discretionary_n_a_':'distribution_type',
                       'current__in_this_fy__or_expired':'current'})

In [8]:
df.sample()

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,eligibility:_school_district,eligibility:_mpo,eligibility:_county_regional_govt__not_mpo,eligibility:_other_local_entity__non_municipality_,eligibility:_state_public_lands,eligibility:_federal_public_lands,eligibility:_tribal_government,eligibility:_non_profit,eligibility:_state_department_,eligibility:_for_profit_companies
56,NaN,Federal Transit Administration (DOT),Metropolitan Transportation Program,Formula funds are apportioned after appropriat...,Missing,Missing,Year of Apportionment plus three years,7.99e+08,States and Metropolitan Planning Organizations,Formula,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   grant_#                                             76 non-null     float64
 1   division                                            77 non-null     object 
 2   grant_program                                       77 non-null     object 
 3   notice_of_funding                                   77 non-null     object 
 4   fy_due_date                                         77 non-null     object 
 5   months_to_apply__c_d_                               77 non-null     object 
 6   fy_cycle                                            77 non-null     object 
 7   funding_amount_available                            77 non-null     float64
 8   agency_eligibility                                  77 non-null     object 
 9   g

In [10]:
df.federal_or_state_funded.value_counts()

Federal    60
State      12
Both        4
Missing     1
Name: federal_or_state_funded, dtype: int64

In [11]:
(
    df
    >>filter(_.federal_or_state_funded=='State')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)

,division,federal_or_state_funded,avg_funds
0,Clean CA,State,2.96e+08
1,DLA,State,1.27e+07
2,DRMT,State,9.16e+07
3,SB1,State,4.78e+08
4,SB1/DLA,State,5.79e+08


In [12]:
chart = (
    alt.Chart(
        (
    df
    >>filter(_.federal_or_state_funded=='State')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [13]:
chart = (
    alt.Chart(
       (
    df
    >>filter(_.federal_or_state_funded=='Federal')
    >> group_by(_.division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [14]:
df.sample(10)

,grant_#,division,grant_program,notice_of_funding,fy_due_date,months_to_apply__c_d_,fy_cycle,funding_amount_available,agency_eligibility,grant_type,...,eligibility:_school_district,eligibility:_mpo,eligibility:_county_regional_govt__not_mpo,eligibility:_other_local_entity__non_municipality_,eligibility:_state_public_lands,eligibility:_federal_public_lands,eligibility:_tribal_government,eligibility:_non_profit,eligibility:_state_department_,eligibility:_for_profit_companies
6,7.0,DRMT,Rural Transit & Intercity Bus 5311 & 5311f (a...,2022-03-01 00:00:00,2022-04-29 00:00:00,1,2 Years,7.81e+08,"Public transit providers, state and local gove...",Formula,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN
52,53.0,Federal Transit Administration (DOT),Ferry Service for Rural Communities,2nd Quarter of 2022,Missing,Missing,TBD,2.00e+09,States,Grant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
23,24.0,DLA,Local Highway Safety Improvement Program (HSIP),2022-05-09 00:00:00,2022-09-12 00:00:00,4,2 Years (sometimes 1),2.28e+08,"City, a County, or a Tribal Government",First Come First Serve Basis,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,5.0,DRMT,5307 Urbanized Area Formula Grants,Missing,Missing,Missing,1 Year,5.38e+09,MPOs and Transit Agencies,Formula,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,58.0,Federal Transit Administration (DOT),Public Transportation on Indian Reservations F...,Formula funds are apportioned after appropriat...,Missing,Missing,Year of Apportionment plus two years\n,1.83e+08,Federally recognized Indian Tribes and Alaska ...,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
9,10.0,DRMT,State of Good Repair,2022-07-01 00:00:00,2021-09-01 00:00:00,Missing,1 Year,1.05e+08,Transit operators,Competitive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,23.0,DLA,Safe Route To School Program (SRTS),"March 16th, 2022",2022-06-15 00:00:00,Missing,2 Years,6.50e+08,"Local, Regional, or State Agencies, Caltrans, ...",Competitive,...,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
50,51.0,Federal Highway Administration (DOT),Congestion Relief Program,TBD,Missing,Missing,Available until expended,2.50e+08,"State, Metropolitan Planning Organization, cit...",Competitive,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
18,19.0,SB1,Caltrans Sustainable Transportation Planning: ...,2022-09-01 00:00:00,2021-10-27 00:00:00,Missing,1 Year,3.45e+07,"MPO, RTPA",Competitive,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,59.0,Federal Transit Administration (DOT),Statewide Transportation Planning,Formula funds are apportioned after appropriat...,Missing,Missing,Year of Apportionment plus three years,1.67e+08,States and Metropolitan Planning Organizations,Formula,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [15]:
df>>group_by(_.division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,division,avg_funds
0,Caltrans - Unknown Division,2.30e+09
1,Clean CA,2.96e+08
2,DLA,1.14e+08
3,DLA,0.00e+00
4,DRMT,1.30e+09
5,Federal Highway Administration (DOT),4.22e+08
6,Federal Railroad Administration (DOT),1.00e+09
7,Federal Transit Administration (DOT),5.40e+08
8,Office of the Secretary (DOT),2.07e+09
9,SB1,8.56e+08


In [16]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='State')
            >> group_by(_.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [17]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='Federal')
            >> group_by(_.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [18]:
df>>group_by(_.grant_type, _.division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,grant_type,division,avg_funds
0,Both,DRMT,6.04e+08
1,Both,SB1,1.17e+08
2,Competitive,Caltrans - Unknown Division,2.30e+09
3,Competitive,DLA,1.35e+08
4,Competitive,DRMT,2.50e+09
5,Competitive,Federal Highway Administration (DOT),4.63e+08
6,Competitive,Federal Railroad Administration (DOT),1.00e+09
7,Competitive,Federal Transit Administration (DOT),4.48e+08
8,Competitive,Office of the Secretary (DOT),2.44e+09
9,Competitive,SB1,1.72e+09


In [19]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='State')
            >> group_by(_.grant_type, _.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "grant_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Grant Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Grant Distribution Type")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)

In [20]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.federal_or_state_funded=='Federal')
            >> group_by(_.grant_type, _.division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("division", title="Division"),
        color=alt.Color(
            "grant_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Grant Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Grant Distribution Type (Nation-wide)")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)